## Crime Analytics

## Communicating Data Science Results 
#### by University of Washington


The Visual crime analysis will analyze criminal incident data from Seattle and San Francisco for the period of June, July, and August 2014.

The analysis will try to answer, mainly visually, to the following questions:

#### 1.How do incidents vary month to month in the Summer 2014 dataset?
#### 2.Which incident types tend to correlate with each other on a day-by-day basis?
#### 3. How do incidents vary by time of day? Which incidents are most common in the evening? During what periods of the day are robberies most common?
#### 4. What can we infer broadly about the differences in crime patterns between Seattle and San Francisco? Does one city tend to have more crime than the other, per capita? Do the relative frequencies of types of incidents change materially between the two cities?
#### 5. Do certain crimes correlate with environmental factors such as temperature? 

In particular, to answer to some of this questions, some external data are required, namely: temperature for summer 2014 and population for both cities.

The temperature data are collected from:
#### National Weather Service website http://w2.weather.gov/climate/index.php?wfo=sew for Seattle, 
and 
#### US Climate Data http://www.usclimatedata.com/climate/san-francisco/california/united-states/usca0987/2014/8 for San Francisco, 
setting the month as the last character into the link provided.



In [24]:
Sys.setlocale("LC_TIME", "English")
options(jupyter.plot_mimetypes = 'image/png')
#setwd("E:/MEGA/Git Repository/datasci_course_materials/assignment6")        
crisan  <- read.csv("sanfrancisco_incidents_summer_2014.csv", header = T)     
crisea  <- read.csv("seattle_incidents_summer_2014.csv", header = T)   
tempsan <- read.csv("SeattleTemp.csv", header = T, sep=";")     
tempsea <- read.csv("SanFranciscoTemp.csv", header = T, sep=";")   
tempsea$TempAvg <- round((tempsea$TempMin+tempsea$TempMax)/2,0)
meanTempSeaAvg  <- round(mean(tempsea$TempAvg),0)
tempsea$TempAvg[tempsea$TempAvg==0] <- meanTempSeaAvg
tempsea$TempMin <- NULL
tempsea$TempMax <- NULL
#str(crisan)
#str(crisea)
crisea$Date <- as.factor(substr(as.character(crisea$Occurred.Date.or.Date.Range.Start), 1, 10))
crisea$Time <- as.factor(substr(as.character(crisea$Occurred.Date.or.Date.Range.Start), 12, 16))
crisea <- merge(x = crisea, y = tempsea, by = "Date")
crisan <- merge(x = crisan, y = tempsan, by = "Date")
crisea$DayOfWeek <- weekdays(as.Date(crisea$Date,'%m/%d/%Y'))


[1] "English_United States.1252"

In [27]:
head(crisea,10)


,Date,RMS.CDW.ID,General.Offense.Number,Offense.Code,Offense.Code.Extension,Offense.Type,Summary.Offense.Code,Summarized.Offense.Description,Date.Reported,Occurred.Date.or.Date.Range.Start,Occurred.Date.Range.End,Hundred.Block.Location,District.Sector,Zone.Beat,Census.Tract.2000,Longitude,Latitude,Location,Month,Year,Time,TempAvg,DayOfWeek
1,06/01/2014,58255,2014172843,5015,2,WARRARR-MISDEMEANOR,5000,WARRANT ARREST,06/01/2014 04:12:00 PM,06/01/2014 04:12:00 PM,,1XX BLOCK OF MERCER ST,Q,Q3,7000.302,-122.3548,47.62458,"(47.624576884, -122.3547512)",6,2014,04:12,54,Sunday
2,06/01/2014,58299,2014904848,2305,0,THEFT-CARPROWL,2300,CAR PROWL,06/01/2014 09:59:00 PM,06/01/2014 04:00:00 PM,06/01/2014 07:35:00 PM,8XX BLOCK OF QUEEN ANNE AV N,Q,Q3,7000.301,-122.3567,47.62675,"(47.62675447, -122.356693132)",6,2014,04:00,54,Sunday
3,06/01/2014,57898,2014173413,2205,0,BURGLARY-NOFORCE-NONRES,2200,BURGLARY,06/02/2014 03:32:00 AM,06/01/2014 02:00:00 AM,06/02/2014 03:30:00 AM,26XX BLOCK OF SW TRENTON ST,F,F2,11402.1,-122.3673,47.52468,"(47.524676734, -122.367264589)",6,2014,02:00,54,Sunday
4,06/01/2014,57820,2014905539,2305,0,THEFT-CARPROWL,2300,CAR PROWL,06/03/2014 10:29:00 AM,06/01/2014 08:00:00 PM,06/02/2014 03:00:00 AM,65XX BLOCK OF 38 AV SW,W,W3,10600.1,-122.3805,47.54413,"(47.544132217, -122.380519165)",6,2014,08:00,54,Sunday
5,06/01/2014,58275,2014172628,2303,0,THEFT-SHOPLIFT,2300,SHOPLIFTING,06/01/2014 12:45:00 PM,06/01/2014 12:41:00 PM,,52XX BLOCK OF CALIFORNIA AV SW,W,W2,10500.3,-122.3869,47.55608,"(47.556078722, -122.386924162)",6,2014,12:41,54,Sunday
6,06/01/2014,999870,2014184601,X,47,PROPERTY FOUND,X,STOLEN PROPERTY,06/10/2014 02:00:00 PM,06/01/2014 12:00:00 AM,06/01/2014 12:00:00 AM,25XX BLOCK OF BEACON AV S,O,O1,10000.5,-122.3129,47.58003,"(47.580032218, -122.312894495)",6,2014,12:00,54,Sunday
7,06/01/2014,58298,2014172219,X,21,DISTURBANCE-OTH,X,DISTURBANCE,06/01/2014 03:49:00 AM,06/01/2014 03:15:00 AM,06/01/2014 03:48:00 AM,12XX BLOCK OF S MAIN ST,G,G1,9000.201,-122.3166,47.60001,"(47.600011025, -122.316550162)",6,2014,03:15,54,Sunday
8,06/01/2014,57423,2014177553,2202,0,BURGLARY-FORCE-RES,2200,BURGLARY,06/05/2014 08:09:00 AM,06/01/2014 03:00:00 PM,06/01/2014 09:30:00 PM,38XX BLOCK OF S MYRTLE ST,S,S1,11001.2,-122.2831,47.53888,"(47.538883654, -122.283139047)",6,2014,03:00,54,Sunday
9,06/01/2014,1000722,2014172945,2903,0,PROPERTY DAMAGE-NON RESIDENTIA,2900,PROPERTY DAMAGE,06/01/2014 07:00:00 PM,06/01/2014 03:00:00 PM,06/01/2014 05:40:00 PM,NE 77 ST / SAND POINT WY NE,L,L3,4000.1,-122.2645,47.68475,"(47.684754597, -122.26447417)",6,2014,03:00,54,Sunday
10,06/01/2014,57591,2014176304,2589,2,NARC-FORGERY-PRESCRIPTION,2500,NARCOTICS,06/04/2014 09:30:00 AM,06/01/2014 09:00:00 AM,,8XX BLOCK OF 3 AV,K,K1,8100.203,-122.333,47.60455,"(47.604553824, -122.333040387)",6,2014,09:00,54,Sunday


In [ ]:
SEATTLE - http://w2.weather.gov/climate/index.php?wfo=sew

PRELIMINARY MONTHLY DATA (CF6) - updated frequently:
Daily weather statistics for the month, including temperatures, precipitation, degree days, wind and sky cover.  In addition, monthly statistics such as average temperatures and departures from normal, degree days, and rainfall are also included.  This product is available for up to 5 years.

PRELIMINARY LOCAL CLIMATOLOGICAL DATA (WS FORM: F-6)

                                          STATION:   SEATTLE-TACOMA WA AIRPORT
                                          MONTH:     JUNE
                                          YEAR:      2014
                                          LATITUDE:   47 27 N
                                          LONGITUDE: 122 18 W

  TEMPERATURE IN F:       :PCPN:    SNOW:  WIND      :SUNSHINE: SKY     :PK WND
================================================================================
1   2   3   4   5  6A  6B    7    8   9   10  11  12  13   14  15   16   17  18
                                     12Z  AVG MX 2MIN
DY MAX MIN AVG DEP HDD CDD  WTR  SNW DPTH SPD SPD DIR MIN PSBL S-S WX    SPD DR
================================================================================

 1  72  51  62   3   3   0 0.00  0.0    0  5.2 10 250   M    M   6        15 220
 2  74  52  63   4   2   0 0.00  0.0    0  5.4 13 210   M    M   4 12     16 200
 3  65  52  59   0   6   0 0.00  0.0    0  7.2 14 220   M    M   8        16 250
 4  67  50  59   0   6   0 0.00  0.0    0  5.8 12 260   M    M   7        16 260
 5  72  50  61   2   4   0 0.00  0.0    0  5.3 10 350   M    M   5        20 250
 6  77  51  64   5   1   0 0.00  0.0    0  7.1 21  40   M    M   4        24  40
 7  76  56  66   6   0   1 0.00  0.0    0  7.0 15 300   M    M   5        20 290
 8  74  54  64   4   1   0 0.00  0.0    0  4.7 13  50   M    M   6 12     15  40
 9  70  56  63   3   2   0    T  0.0    0  8.0 16  30   M    M   8        20  30
10  68  54  61   1   4   0 0.00  0.0    0  6.4 15  40   M    M   8        18  40
11  75  52  64   4   1   0 0.00  0.0    0  6.0 14 290   M    M   6        16 300
12  71  54  63   3   2   0 0.07  0.0    0  9.0 23 240   M    M   8        31 230
13  60  52  56  -4   9   0 0.25  0.0    0 11.2 22 220   M    M  10 1      31 220
14  64  53  59  -2   6   0 0.00  0.0    0  7.2 17 190   M    M  10        22 190
15  65  50  58  -3   7   0 0.02  0.0    0  8.1 22  50   M    M  10 1      25  50
16  64  48  56  -5   9   0 0.14    M    0  5.4 13 210   M    M   8 1      16 210
17  64  50  57  -4   8   0 0.05  0.0    0  6.6 12 230   M    M   9 1      16 210
18  66  52  59  -2   6   0 0.00  0.0    0  6.0 12 210   M    M   8        18 180
19  78  53  66   5   0   1 0.03  0.0    0  8.2 18 240   M    M   7        25 230
20  68  50  59  -3   6   0 0.01  0.0    0  7.5 17  30   M    M   7        23 230
21  72  51  62   0   3   0 0.00  0.0    0  8.1 15  20   M    M   4        20 300
22  77  52  65   3   0   0 0.00  0.0    0  6.0 13  20   M    M   6        16 360
23  77  56  67   5   0   2    T  0.0    0  5.6 14 250   M    M   8        18 230
24  76  58  67   5   0   2 0.00  0.0    0  5.5 13 190   M    M   7        21 260
25  79  57  68   6   0   3 0.00  0.0    0  5.4 14 360   M    M   7        17 280
26  70  58  64   1   1   0    T  0.0    0  9.1 17 230   M    M   8        22 240
27  70  57  64   1   1   0 0.07  0.0    0 10.1 22 240   M    M   9 1      30 230
28  68  56  62  -1   3   0 0.09  0.0    0  9.7 17 220   M    M   8 1      22 230
29  69  55  62  -1   3   0    T  0.0    0  7.2 16 210   M    M   6        21 200
30  78  55  67   4   0   2 0.00  0.0    0  9.9 15 330   M    M   3        22 340

PRELIMINARY LOCAL CLIMATOLOGICAL DATA (WS FORM: F-6)

                                          STATION:   SEATTLE-TACOMA WA AIRPORT
                                          MONTH:     JULY
                                          YEAR:      2014
                                          LATITUDE:   47 27 N
                                          LONGITUDE: 122 18 W

  TEMPERATURE IN F:       :PCPN:    SNOW:  WIND      :SUNSHINE: SKY     :PK WND
================================================================================
1   2   3   4   5  6A  6B    7    8   9   10  11  12  13   14  15   16   17  18
                                     12Z  AVG MX 2MIN
DY MAX MIN AVG DEP HDD CDD  WTR  SNW DPTH SPD SPD DIR MIN PSBL S-S WX    SPD DR
================================================================================

 1  94  60  77  13   0  12 0.00  0.0    0  7.9 15 350   M    M   4        18  10
 2  81  58  70   6   0   5 0.00  0.0    0  8.1 16 210   M    M   6        21 230
 3  71  57  64   0   1   0 0.00  0.0    0  6.9 12 230   M    M   8        16 330
 4  75  57  66   2   0   1 0.00  0.0    0  8.0  M 360   M    M   7         M 330
 5  76  56  66   2   0   1 0.00  0.0    0  5.0 14 200   M    M   8 1      18 230
 6  84  59  72   8   0   7 0.00  0.0    0  6.7 17 340   M    M   5        21 330
 7  81  64  73   8   0   8 0.00  0.0    0  9.1 15 340   M    M   5 1      33 340
 8  86  60  73   8   0   8 0.00  0.0    0  7.8 16 350   M    M   2        21 350
 9  80  57  69   4   0   4 0.00  0.0    0  5.1 15 250   M    M   2        25 320
10  84  55  70   5   0   5 0.00  0.0    0  5.0  9  10   M    M   2 1      16 270
11  88  59  74   9   0   9 0.00  0.0    0  5.0 11  35   M    M   2        15  35
12  90  62  76  11   0  11 0.00  0.0    0  4.8 12  20   M    M   3        14  20
13  85  59  72   6   0   7    T  0.0    0  5.9 17 360   M    M   5 3      21 350
14  82  59  71   5   0   6 0.00  0.0    0   M  10 250   M    M   3        11 250
15  88  57  73   7   0   8 0.00  0.0    0  5.2 15 360   M    M   3 12     24 210
16  88  58  73   7   0   8 0.00  0.0    0  5.3 13 200   M    M   0        21 100
17  80  57  69   3   0   4 0.00  0.0    0  8.2 18 280   M    M   6        26 170
18  75  53  64  -2   1   0 0.00  0.0    0  6.3 14 260   M    M   6        18 240
19  78  59  69   3   0   4 0.00  0.0    0 12.0 28 220   M    M   9 1      36 220
20  67  58  63  -3   2   0 0.00  0.0    0  6.3 20 320   M    M  10        26 300
21  75  56  66   0   0   1 0.00  0.0    0  4.8 12 260   M    M   6        15 220
22  70  56  63  -4   2   0 0.01  0.0    0  2.4  9 200   M    M   7 1      16 220
23  66  56  61  -6   4   0 0.76  0.0    0  7.3 14 150   M    M   9 13     25 210
24  69  55  62  -5   3   0    T  0.0    0 10.4 20 210   M    M   8        25 210
25  73  54  64  -3   1   0 0.00  0.0    0  6.0 13  20   M    M   5        17 250
26  79  56  68   1   0   3 0.00  0.0    0  8.0 16  20   M    M   1        17  20
27  83  59  71   4   0   6 0.00  0.0    0  9.1 15 350   M    M   1        18 360
28  87  59  73   6   0   8 0.00  0.0    0  8.2 15  40   M    M   0        22 360
29  86  60  73   6   0   8 0.00  0.0    0  6.3 17  10   M    M   0        21 350
30  85  58  72   5   0   7 0.00  0.0    0  7.6 17  40   M    M   2        23 350
31  87  64  76   9   0  11 0.00  0.0    0  9.1 15  40   M    M   2         M  70


PRELIMINARY LOCAL CLIMATOLOGICAL DATA (WS FORM: F-6)

                                          STATION:   SEATTLE-TACOMA WA AIRPORT
                                          MONTH:     AUGUST
                                          YEAR:      2014
                                          LATITUDE:   47 27 N
                                          LONGITUDE: 122 18 W

  TEMPERATURE IN F:       :PCPN:    SNOW:  WIND      :SUNSHINE: SKY     :PK WND
================================================================================
1   2   3   4   5  6A  6B    7    8   9   10  11  12  13   14  15   16   17  18
                                     12Z  AVG MX 2MIN
DY MAX MIN AVG DEP HDD CDD  WTR  SNW DPTH SPD SPD DIR MIN PSBL S-S WX    SPD DR
================================================================================

 1  84  59  72   5   0   7    T  0.0    0  7.3  M   M   M    M   6         M  M
 2  85  60  73   6   0   8 0.02  0.0    0  3.8 10  20   M    M   6 35      M  M
 3  89  58  74   7   0   9 0.00  0.0    0  5.9 12 360   M    M   1        16 330
 4  91  61  76   9   0  11 0.00  0.0    0  5.8 13 350   M    M   1        15  10
 5  77  57  67   0   0   2 0.00  0.0    0  6.1 13 240   M    M   4        16 210
 6  79  59  69   2   0   4 0.00  0.0    0  4.8 10 250   M    M   6 1      12 320
 7  78  56  67   0   0   2 0.00  0.0    0  5.3  9 190   M    M   5 1      14 210
 8  78  56  67   0   0   2 0.00  0.0    0  6.5 17  40   M    M   5        22 230
 9  81  60  71   4   0   6 0.00  0.0    0  9.1 16  40   M    M   4        21 340
10  87  57  72   5   0   7 0.00  0.0    0  7.7 14 360   M    M   1        21 350
11  96  64  80  13   0  15 0.02  0.0    0  5.8 18  70   M    M   4 3      22  70
12  81  63  72   5   0   7 0.50  0.0    0  7.0 18  60   M    M   9 1      33 350
13  74  59  67   1   0   2 0.85  0.0    0  6.1  8 220   M    M   9 1       8 220
14  70  63  67   1   0   2    T  0.0    0  1.4  5  90   M    M   9 12      7  80
15  76  62  69   3   0   4 0.04  0.0    0  3.4  9 280   M    M   9 1      16 180
16  78  60  69   3   0   4 0.00  0.0    0  4.9 15 290   M    M   7        17 320
17  82  59  71   5   0   6 0.00  0.0    0  6.2 13 360   M    M   3         M  M
18  85  60  73   7   0   8 0.00  0.0    0  7.4 15 340   M    M   2         M  M
19  81  60  71   5   0   6 0.00  0.0    0  5.4 16 210   M    M   5        20 210
20  71  57  64  -2   1   0 0.00  0.0    0  8.1 13 230   M    M   7        17 220
21  70  52  61  -5   4   0 0.00  0.0    0  3.9 10 220   M    M   5        15 220
22  75  56  66   0   0   1 0.00  0.0    0  6.5 16 330   M    M   2        21  20
23  82  57  70   4   0   5 0.00  0.0    0  4.4 12 280   M    M   2        16 280
24  77  56  67   1   0   2 0.00  0.0    0  5.2 10  10   M    M   4 12     15 270
25  84  58  71   6   0   6 0.00  0.0    0  4.4 15 360   M    M   2        21  10
26  88  60  74   9   0   9 0.00  0.0    0  4.0 10  20   M    M   0         M  M
27  84  61  73   8   0   8 0.00  0.0    0  3.6 12  30   M    M   2        22 100
28  74  58  66   1   0   1 0.00  0.0    0  5.2 12 180   M    M   7        16 180
29  73  59  66   1   0   1    T  0.0    0  7.5 16 210   M    M   8        20 200
30  64  59  62  -3   3   0 0.33  0.0    0  4.8 13 120   M    M   9 1      15 130
31  70  57  64  -1   1   0 0.05  0.0    0  4.2 10 210   M    M  10 1      14 190

In [ ]:
http://www.usclimatedata.com/climate/san-francisco/california/united-states/usca0987/2014/8


Weather history San Francisco june 2014

Day	High

(°F)	Low

(°F)	Precip.

(inch)	Snow

(inch)	Snow
depth
(inch)
1 jun 2014	55.9	51.1	0.00	-	-
2 jun 2014	-	-	0.00	-	-
3 jun 2014	57.0	50.0	0.00	-	-
4 jun 2014	55.9	51.1	0.00	-	-
5 jun 2014	55.9	50.0	0.00	-	-
6 jun 2014	59.0	52.0	0.00	-	-
7 jun 2014	59.0	52.0	0.00	-	-
8 jun 2014	59.0	51.1	0.00	-	-
9 jun 2014	60.1	51.1	0.00	-	-
10 jun 2014	60.1	51.1	0.00	-	-
11 jun 2014	59.0	55.9	0.00	-	-
12 jun 2014	57.9	52.0	0.00	-	-
13 jun 2014	63.0	54.0	0.00	-	-
14 jun 2014	62.1	53.1	0.00	-	-
15 jun 2014	60.1	53.1	0.00	-	-
16 jun 2014	57.9	53.1	0.00	-	-
17 jun 2014	57.9	52.0	0.00	-	-
18 jun 2014	55.9	52.0	0.00	-	-
19 jun 2014	62.1	51.1	0.00	-	-
20 jun 2014	62.1	51.1	0.00	-	-
21 jun 2014	60.1	54.0	0.00	-	-
22 jun 2014	57.9	54.0	0.00	-	-
23 jun 2014	59.0	53.1	0.00	-	-
24 jun 2014	60.1	55.0	0.00	-	-
25 jun 2014	-	-	0.00	-	-
26 jun 2014	-	-	0.00	-	-
27 jun 2014	-	-	0.00	-	-
28 jun 2014	64.0	54.0	0.00	-	-
29 jun 2014	62.1	54.0	0.00	-	-
30 jun 2014	62.1	55.0	0.00	-	-

Weather history San Francisco july 2014

Day	High

(°F)	Low

(°F)	Precip.

(inch)	Snow

(inch)	Snow
depth
(inch)
1 jul 2014	62.1	55.0	0.00	-	-
2 jul 2014	62.1	55.0	0.00	-	-
3 jul 2014	57.9	55.0	0.00	-	-
4 jul 2014	59.0	54.0	0.00	-	-
5 jul 2014	60.1	54.0	0.00	-	-
6 jul 2014	60.1	53.1	0.00	-	-
7 jul 2014	60.1	53.1	0.00	-	-
8 jul 2014	59.0	55.0	0.00	-	-
9 jul 2014	61.0	55.9	0.00	-	-
10 jul 2014	64.0	57.0	0.00	-	-
11 jul 2014	64.9	55.9	0.00	-	-
12 jul 2014	64.9	55.9	0.00	-	-
13 jul 2014	64.9	55.0	0.00	-	-
14 jul 2014	64.9	55.0	0.00	-	-
15 jul 2014	64.9	55.0	0.00	-	-
16 jul 2014	64.9	55.0	0.00	-	-
17 jul 2014	69.1	62.1	0.00	-	-
18 jul 2014	-	-	0.00	-	-
19 jul 2014	64.0	59.0	0.00	-	-
20 jul 2014	64.9	59.0	0.00	-	-
21 jul 2014	66.0	61.0	0.00	-	-
22 jul 2014	66.0	60.1	0.00	-	-
23 jul 2014	66.9	60.1	0.00	-	-
24 jul 2014	-	-	0.00	-	-
25 jul 2014	69.1	59.0	0.00	-	-
26 jul 2014	68.0	60.1	0.00	-	-
27 jul 2014	64.0	57.9	0.00	-	-
28 jul 2014	62.1	57.9	0.00	-	-
29 jul 2014	-	-	0.00	-	-
30 jul 2014	-	-	0.00	-	-
31 jul 2014	-	-	0.00	-	-


Weather history San Francisco august 2014

Day	High

(°F)	Low

(°F)	Precip.

(inch)	Snow

(inch)	Snow
depth
(inch)
1 aug 2014	62.1	52.0	0.00	-	-
2 aug 2014	62.1	55.9	0.00	-	-
3 aug 2014	62.1	51.1	0.00	-	-
4 aug 2014	62.1	55.9	0.00	-	-
5 aug 2014	63.0	55.9	0.00	-	-
6 aug 2014	64.0	55.9	0.00	-	-
7 aug 2014	64.0	55.9	0.00	-	-
8 aug 2014	64.0	55.9	0.00	-	-
9 aug 2014	64.0	55.9	0.00	-	-
10 aug 2014	64.0	57.0	0.00	-	-
11 aug 2014	-	-	0.00	-	-
12 aug 2014	66.0	55.9	0.00	-	-
13 aug 2014	66.0	57.0	0.00	-	-
14 aug 2014	66.0	57.0	0.00	-	-
15 aug 2014	64.0	57.9	0.00	-	-
16 aug 2014	64.0	57.9	0.00	-	-
17 aug 2014	64.0	57.0	0.00	-	-
18 aug 2014	64.0	57.0	0.00	-	-
19 aug 2014	64.9	57.0	0.00	-	-
20 aug 2014	64.9	61.0	0.00	-	-
21 aug 2014	64.9	60.1	0.00	-	-
22 aug 2014	64.9	59.0	0.00	-	-
23 aug 2014	66.0	60.1	0.00	-	-
24 aug 2014	66.0	60.1	0.00	-	-
25 aug 2014	66.0	61.0	0.00	-	-
26 aug 2014	66.9	60.1	0.00	-	-
27 aug 2014	66.9	60.1	0.00	-	-
28 aug 2014	66.9	53.1	0.00	-	-
29 aug 2014	66.9	59.0	0.00	-	-
30 aug 2014	66.9	60.1	0.00	-	-
31 aug 2014	66.9	60.1	0.00	-	-


In [15]:
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="INJURY"] <- "ASSAULT"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="HOMICIDE"] <- "ASSAULT"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="BURGLARY-SECURE PARKING-RES"] <- "BURGLARY"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="TRESPASS"] <- "BURGLARY"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="PUBLIC NUISANCE"] <- "DISORDERLY CONDUCT"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="THREATS"] <- "DISORDERLY CONDUCT"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="DUI"] <- "DRIVING UNDER THE INFLUENCE"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="NARCOTICS"] <- "DRUG/NARCOTIC"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="TRAFFIC"] <- "DRUG/NARCOTIC"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="EMBEZZLE"] <- "EMBEZZLEMENT"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="COUNTERFEIT"] <- "FORGERY/COUNTERFEITING"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="FORGERY"]   <- "FORGERY/COUNTERFEITING"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="MAIL THEFT"] <- "LARCENY/THEFT"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="PICKPOCKET"] <- "LARCENY/THEFT"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="PURSE SNATCH"] <- "LARCENY/THEFT"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="SHOPLIFTING"] <- "LARCENY/THEFT"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="STOLEN PROPERTY"] <- "LARCENY/THEFT"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="THEFT OF SERVICES"] <- "LARCENY/THEFT"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="LIQUOR VIOLATION"] <- "LIQUOR LAWS"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="LOST PROPERTY"] <- "NON-CRIMINAL"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="RECOVERED PROPERTY"] <- "NON-CRIMINAL"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="ANIMAL COMPLAINT"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="BIAS INCIDENT"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="CAR PROWL"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="DISPUTE"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="DISTURBANCE"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="FALSE REPORT"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="FIREWORK"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="[INC - CASE DC USE ONLY]"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="ILLEGAL DUMPING"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="OBSTRUCT"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="OTHER PROPERTY"] <- "OTHER OFFENSES"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="VIOLATION OF COURT ORDER"] <- "OTHER OFFENSES"


levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="PORNOGRAPHY"] <- "PORNOGRAPHY/OBSCENE MAT"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="ELUDING"] <- "RUNAWAY"
levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="ESCAPE"] <- "RUNAWAY"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="PROPERTY DAMAGE"] <- "VANDALISM"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="BIKE THEFT"] <- "VEHICLE THEFT"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="WARRANT ARREST"] <- "WARRANTS"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="WEAPON"] <- "WEAPON LAWS"

levels(crisea$Summarized.Offense.Description)[levels(crisea$Summarized.Offense.Description)=="RECKLESS BURNING"] <- "ARSON"

levels(crisan$Category)[levels(crisan$Category)=="BRIBERY"] <- "EMBEZZLEMENT"
levels(crisan$Category)[levels(crisan$Category)=="EXTORTION"] <- "EMBEZZLEMENT"
levels(crisan$Category)[levels(crisan$Category)=="DRUNKENNESS"] <- "DISORDERLY CONDUCT"
levels(crisan$Category)[levels(crisan$Category)=="FAMILY OFFENSES"] <- "DISORDERLY CONDUCT"
levels(crisan$Category)[levels(crisan$Category)=="SUSPICIOUS OCC"] <- "DISORDERLY CONDUCT"
levels(crisan$Category)[levels(crisan$Category)=="KIDNAPPING"] <- "ASSAULT"
levels(crisan$Category)[levels(crisan$Category)=="GAMBLING"] <- "SECONDARY CODES"
levels(crisan$Category)[levels(crisan$Category)=="LOITERING"] <- "SECONDARY CODES"
levels(crisan$Category)[levels(crisan$Category)=="MISSING PERSON"] <- "SECONDARY CODES"
levels(crisan$Category)[levels(crisan$Category)=="SECONDARY CODES"] <- "OTHER OFFENSES"
levels(crisan$Category)[levels(crisan$Category)=="STOLEN PROPERTY"] <- "OTHER OFFENSES"
levels(crisan$Category)[levels(crisan$Category)=="SUICIDE"] <- "OTHER OFFENSES"
levels(crisan$Category)[levels(crisan$Category)=="TRESPASS"] <- "RUNAWAY"

sort(levels(crisea$Summarized.Offense.Description)) == sort(levels(crisan$Category))




[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE

In [172]:
#head(crisea[crisea$Summarized.Offense.Description=="ASSAULT",])
#crisea[crisea$RMS.CDW.ID==1047623,]
#crisea[crisea$RMS.CDW.ID==1067822,]
min(as.character(crisea$Date))
max(as.character(crisea$Date))





[1] "06/01/2014"

[1] "08/31/2014"

,Date,TempAvg
1,06/01/2014,62
2,06/02/2014,63
3,06/03/2014,59
4,06/04/2014,59
5,06/05/2014,61
6,06/06/2014,64


,RMS.CDW.ID,General.Offense.Number,Offense.Code,Offense.Code.Extension,Offense.Type,Summary.Offense.Code,Summarized.Offense.Description,Date.Reported,Occurred.Date.or.Date.Range.Start,Occurred.Date.Range.End,Hundred.Block.Location,District.Sector,Zone.Beat,Census.Tract.2000,Longitude,Latitude,Location,Month,Year,Date,Time
1625,1047623,2014288580,X,36,INJURY - ACCIDENTAL,X,ASSAULT,08/29/2014 04:53:00 PM,08/29/2014 02:50:00 PM,,143XX BLOCK OF STONE AV N,N,N1,300.3003,0,0,"(0.0, 0.0)",8,2014,08/29/2014,02:50


,RMS.CDW.ID,General.Offense.Number,Offense.Code,Offense.Code.Extension,Offense.Type,Summary.Offense.Code,Summarized.Offense.Description,Date.Reported,Occurred.Date.or.Date.Range.Start,Occurred.Date.Range.End,Hundred.Block.Location,District.Sector,Zone.Beat,Census.Tract.2000,Longitude,Latitude,Location,Month,Year,Date,Time
209,1067822,2014335684,1305,0,ASSLT-AGG-WEAPON,1300,ASSAULT,10/07/2014 04:15:00 PM,07/20/2014 08:30:00 PM,,S MAIN ST / OCCIDENTAL AV S,K,K3,9200.201,-122.3329,47.60004,"(47.600040755, -122.332882687)",7,2014,07/20/2014,08:30


In [16]:
head(crisea)
head(crisan)


,Date,RMS.CDW.ID,General.Offense.Number,Offense.Code,Offense.Code.Extension,Offense.Type,Summary.Offense.Code,Summarized.Offense.Description,Date.Reported,Occurred.Date.or.Date.Range.Start,Occurred.Date.Range.End,Hundred.Block.Location,District.Sector,Zone.Beat,Census.Tract.2000,Longitude,Latitude,Location,Month,Year,Time,TempAvg
1,06/01/2014,58255,2014172843,5015,2,WARRARR-MISDEMEANOR,5000,WARRANTS,06/01/2014 04:12:00 PM,06/01/2014 04:12:00 PM,,1XX BLOCK OF MERCER ST,Q,Q3,7000.302,-122.3548,47.62458,"(47.624576884, -122.3547512)",6,2014,04:12,54
2,06/01/2014,58299,2014904848,2305,0,THEFT-CARPROWL,2300,OTHER OFFENSES,06/01/2014 09:59:00 PM,06/01/2014 04:00:00 PM,06/01/2014 07:35:00 PM,8XX BLOCK OF QUEEN ANNE AV N,Q,Q3,7000.301,-122.3567,47.62675,"(47.62675447, -122.356693132)",6,2014,04:00,54
3,06/01/2014,57898,2014173413,2205,0,BURGLARY-NOFORCE-NONRES,2200,BURGLARY,06/02/2014 03:32:00 AM,06/01/2014 02:00:00 AM,06/02/2014 03:30:00 AM,26XX BLOCK OF SW TRENTON ST,F,F2,11402.1,-122.3673,47.52468,"(47.524676734, -122.367264589)",6,2014,02:00,54
4,06/01/2014,57820,2014905539,2305,0,THEFT-CARPROWL,2300,OTHER OFFENSES,06/03/2014 10:29:00 AM,06/01/2014 08:00:00 PM,06/02/2014 03:00:00 AM,65XX BLOCK OF 38 AV SW,W,W3,10600.1,-122.3805,47.54413,"(47.544132217, -122.380519165)",6,2014,08:00,54
5,06/01/2014,58275,2014172628,2303,0,THEFT-SHOPLIFT,2300,LARCENY/THEFT,06/01/2014 12:45:00 PM,06/01/2014 12:41:00 PM,,52XX BLOCK OF CALIFORNIA AV SW,W,W2,10500.3,-122.3869,47.55608,"(47.556078722, -122.386924162)",6,2014,12:41,54
6,06/01/2014,999870,2014184601,X,47,PROPERTY FOUND,X,LARCENY/THEFT,06/10/2014 02:00:00 PM,06/01/2014 12:00:00 AM,06/01/2014 12:00:00 AM,25XX BLOCK OF BEACON AV S,O,O1,10000.5,-122.3129,47.58003,"(47.580032218, -122.312894495)",6,2014,12:00,54


,Date,IncidntNum,Category,Descript,DayOfWeek,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,TempAvg
1,06/01/2014,140457905,ASSAULT,THREATS AGAINST LIFE,Sunday,23:40,TENDERLOIN,"ARREST, BOOKED",MARKET ST / HYDE ST,-122.4147,37.77872,"(37.778719262789, -122.414743835382)",1.404579e+13,62
2,06/01/2014,140457905,OTHER OFFENSES,CONSPIRACY,Sunday,23:40,TENDERLOIN,"ARREST, BOOKED",MARKET ST / HYDE ST,-122.4147,37.77872,"(37.778719262789, -122.414743835382)",1.404579e+13,62
3,06/01/2014,140457814,OTHER OFFENSES,TRAFFIC VIOLATION,Sunday,23:37,MISSION,"ARREST, CITED",FOLSOM ST / 16TH ST,-122.4153,37.76532,"(37.765316135635, -122.415308388816)",1.404578e+13,62
4,06/01/2014,140461796,VEHICLE THEFT,STOLEN AUTOMOBILE,Sunday,23:30,TENDERLOIN,NONE,LEAVENWORTH ST / TURK ST,-122.4141,37.78279,"(37.7827931071006, -122.414056291891)",1.404618e+13,62
5,06/01/2014,140457977,OTHER OFFENSES,MISSING ADULT,Sunday,23:30,RICHMOND,NONE,1700 Block of PIERCE ST,-122.4366,37.78615,"(37.7861520898411, -122.43658540753)",1.40458e+13,62
6,06/01/2014,140457977,OTHER OFFENSES,FOUND PERSON,Sunday,23:30,RICHMOND,NONE,1700 Block of PIERCE ST,-122.4366,37.78615,"(37.7861520898411, -122.43658540753)",1.40458e+13,62
